MACHINE TRANSLATION FOR KISWAHILI SAWA CORPUS USING JOEY NMT 

In [7]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "sw" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag


In [2]:
! pip3 install pandas
import pandas as pd

  Using cached https://files.pythonhosted.org/packages/bb/71/8f53bdbcbc67c912b888b40def255767e475402e9df64050019149b1a943/pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3d/fc/4763e5f17ac6e7e7d55f377cde859ca1c5d5ac624441ab45315bc578aa9e/numpy-1.18.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c6fae16075da282f5058082d4cb10c6c5c1dba/six-1.14.0-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
data = pd.read_csv('sawa_corpus.csv',engine='python',names=['source_sentence', 'target_sentence'])

In [4]:
print(data.head)

<bound method NDFrame.head of                                           source_sentence  \
0                                 Chairman�s Message \r\n   
1       Dear Shareholders It is my pleasure to present...   
2       The greater than three times oversubscription ...   
3       I therefore take this opportunity, on behalf o...   
4       The rate of economic growth in our country in ...   
...                                                   ...   
272539                                      infatuate\r\n   
272540                                          craze\r\n   
272541                     vaunt fool flatter oneself\r\n   
272542                             sink into oblivion\r\n   
272543                              be foolish stupid\r\n   

                                          target_sentence  
0                              Taarifa ya Mwenyekiti \r\n  
1       WapendwaWenyehisa Nifuraha yangu kuu kuwasilis...  
2       Uuzaji wa zaidi ya mara tatu wa hisa zetu kwa ...

Pre-processing and export
It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [5]:
seed = 42  # Random seed for shuffling.
# drop duplicate translations
df_pp = data.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Getting the test data from the masakhane repo official test data

In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-04-23 18:39:42--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.3’

test.en-any.en.3    100%[===================>] 271.28K  --.-KB/s    in 0.009s  

2020-04-23 18:39:43 (29.2 MB/s) - ‘test.en-any.en.3’ saved [277791/277791]

--2020-04-23 18:39:43--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-sw.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


In [7]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [8]:
! pip3 install fuzzywuzzy

  Using cached https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
#! sudo apt-get install python3.6-dev
! pip3 install python-Levenshtein

  Using cached https://files.pythonhosted.org/packages/a0/df/635cdb901ee4a8a42ec68e480c49f85f4c59e8816effbf57d9e6ee8b3588/setuptools-46.1.3-py3-none-any.whl
You are using pip version 8.1.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
#! pip3 install fuzzywuzzy
#! pip3 install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

00:00:00.03 0.00 percent complete
00:00:12.87 1.11 percent complete
00:00:25.83 2.22 percent complete
00:00:38.33 3.33 percent complete
00:00:51.02 4.44 percent complete
00:01:03.82 5.56 percent complete


']


00:01:16.44 6.67 percent complete
00:01:29.09 7.78 percent complete
00:01:41.83 8.89 percent complete
00:01:53.80 10.00 percent complete
00:02:06.28 11.11 percent complete


']


00:02:19.70 12.22 percent complete
00:02:33.24 13.33 percent complete
00:02:45.92 14.44 percent complete
00:02:59.63 15.56 percent complete
00:03:12.26 16.67 percent complete
00:03:25.67 17.78 percent complete
00:03:38.01 18.89 percent complete
00:03:50.14 20.00 percent complete
00:04:02.47 21.11 percent complete
00:04:15.51 22.22 percent complete
00:04:29.31 23.33 percent complete
00:04:42.02 24.44 percent complete
00:04:54.26 25.56 percent complete
00:05:07.20 26.67 percent complete


...

00:05:32.67 28.89 percent complete
00:05:45.50 30.00 percent complete
00:05:57.77 31.11 percent complete
00:06:10.33 32.22 percent complete
00:06:22.82 33.33 percent complete
00:06:35.96 34.44 percent complete
00:06:49.31 35.56 percent complete
00:07:02.43 36.67 percent complete
00:07:15.11 37.78 percent complete
00:07:28.23 38.89 percent complete
00:07:40.81 40.00 percent complete
00:07:53.32 41.11 percent complete
00:08:06.72 42.22 percent complete
00:08:19.72 43.33 percent complete
00:08:32.77 44.44 percent complete
00:08:45.16 45.56 percent complete
00:08:58.33 46.67 percent complete
00:09:11.42 47.78 percent complete
00:09:24.19 48.89 percent complete
00:09:36.84 50.00 percent complete
00:09:49.79 51.11 percent complete
00:10:02.10 52.22 percent complete
00:10:14.97 53.33 percent complete
00:10:28.03 54.45 percent complete
00:10:40.75 55.56 percent complete


']


00:10:54.16 56.67 percent complete
00:11:07.13 57.78 percent complete
00:11:19.98 58.89 percent complete
00:11:32.62 60.00 percent complete
00:11:45.04 61.11 percent complete
00:11:57.95 62.22 percent complete
00:12:11.03 63.33 percent complete
00:12:23.83 64.45 percent complete
00:12:36.69 65.56 percent complete
00:12:49.67 66.67 percent complete
00:13:02.23 67.78 percent complete
00:13:15.22 68.89 percent complete
00:13:28.02 70.00 percent complete
00:13:40.50 71.11 percent complete
00:13:53.06 72.22 percent complete
00:14:06.27 73.33 percent complete
00:14:18.51 74.45 percent complete


']


00:14:31.31 75.56 percent complete
00:14:44.44 76.67 percent complete
00:14:57.98 77.78 percent complete
00:15:11.06 78.89 percent complete
00:15:23.95 80.00 percent complete
00:15:36.73 81.11 percent complete
00:15:49.40 82.22 percent complete
00:16:02.48 83.33 percent complete
00:16:15.09 84.45 percent complete
00:16:27.80 85.56 percent complete
00:16:40.50 86.67 percent complete
00:16:53.57 87.78 percent complete
00:17:05.90 88.89 percent complete
00:17:18.78 90.00 percent complete
00:17:31.48 91.11 percent complete
00:17:44.91 92.22 percent complete
00:17:57.67 93.33 percent complete
00:18:11.20 94.45 percent complete
00:18:24.00 95.56 percent complete
00:18:37.10 96.67 percent complete
00:18:50.06 97.78 percent complete
00:19:03.62 98.89 percent complete


In [11]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
lobster 

EZK:34:7 Therefore, you shepherds, hear the word of the LORD; 

hush

someone else wore kangas with  cashew apples, they made her look good.

the Vatican under the leadership of the new Pope Benedict XVI has criticised a new law in Spain that will allow single sex couples to marry


==> train.sw <==
kambakoche

EZK:34:7 Basi, enyi wachungaji, lisikieni neno la Bwana; 

uss

fulani mwengine alivaa kanga za bibo, zimempendeza

Vatican chini ya uongozi wa papa mpya Benedict wa 16 imeishutumu sheria mpya ya Uhispania itakayowarushusu wapenzi wa jinsia moja kuoana

==> dev.en <==
span 

administer

Then as to Samood, they were destroyed by an excessively severe punishment.

You do not believe but say, 

GEN:2:6 But there went up a mist from the earth, and watered the whole face of the ground. 


==> dev.sw <==
shibiri

amuru

Basi Thamudi waliangamizwa kwa balaa kubwa mno.

Hamjaamini, lakini semeni:

GEN:2:6 ukungu ukapanda katika nchi, ukatia maji juu ya uso wot

Installatiom of JOEY NMT

In [10]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /home/freshiasackey_gmail_com/joeynmt
  Using cached https://files.pythonhosted.org/packages/ba/90/8a24e6220cfcf6a3a0162535d5b926e774117e384ff921908e07e4c92bda/Pillow-7.1.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3d/fc/4763e5f17ac6e7e7d55f377cde859ca1c5d5ac624441ab45315bc578aa9e/numpy-1.18.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a0/df/635cdb901ee4a8a42ec68e480c49f85f4c59e8816effbf57d9e6ee8b3588/setuptools-46.1.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/13/70/54e9fb010fe1547bc4774716f11ececb81ae5b306c05f090f4461ee13205/torch-1.5.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/de/f0/96fb2e0412ae9692dbf400e5b04432885f677ad6241c088ccc5fe7724d69/tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhos

  Using cached https://files.pythonhosted.org/packages/e5/b0/c121fd1fa3419ea9bfd55c7f9c4fedfec5143208d8c7ad3ce3db6c623c21/isort-4.3.21-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/06/cafdd44889200e5438b897388f3075b52a8ef01f28a17366d91de0fa2d05/h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cc/94/5f7079a0e00bd6863ef8f1da638721e9da21e5bacee597595b318f71d62e/Werkzeug-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ab/c4/ba46d44855e6eb1770a12edace5a165a0c6de13349f592b9036257f3c3d3/Markdown-3.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packag

Preprocessing the Data into Subword BPE Tokens
One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization (Sennrich, 2015) .

It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages (Sennrich, 2019) (Martinus, 2019)

Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by (Sennrich, 2019). You do not need to change anything. Simply running the below will be suitable.

In [4]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# # Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp bpe.codes.4000 "$gdrive_path"
# ! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Swahili Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.sw	     test.en-any.en    train.bpe.en
dev.bpe.en	test.bpe.en  test.en-any.en.1  train.bpe.sw
dev.bpe.sw	test.bpe.sw  test.en-any.en.2  train.en
dev.en		test.en      test.sw	       train.sw
BPE Swahili Sentences
N@@ ga@@ o kubwa ya imani ( Ta@@ zama fungu la 1@@ 2 hadi 1@@ 4 )
K@@ ofi@@ a ya chu@@ ma ya woko@@ vu ( Ta@@ zama fungu la 1@@ 5 hadi 1@@ 8 )
Nime@@ ta@@ mbua kwamba watu hu@@ i@@ ti@@ kia vi@@ zuri wana@@ po@@ ona uki@@ zungu@@ m@@ zia habari za Bi@@ b@@ lia kwa sha@@ u@@ ku na una@@ po@@ fanya yote una@@ yo@@ weza kuwa@@ saidia . ”
U@@ panga wa roho ( Ta@@ zama fungu la 1@@ 9 na 2@@ 0 )
Kwa msa@@ ada wa Yeho@@ v@@ a tuna@@ weza ku@@ simama i@@ mara na ku@@ mp@@ inga !
Combined BPE Vocab
ù@@
harao@@
 
>@@
fles@@
}@@
^@@
|
berna@@
ù


Creating the JoeyNMT Config
JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

We used Transformer architecture
We set our dropout to reasonably high: 0.3 (recommended in (Sennrich, 2019))
Things worth playing with:

The batch size (also recommended to change for low-resourced languages)
The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
The decoder options (beam_size, alpha)
Evaluation metrics (BLEU versus Crhf4)

In [10]:
# More experiments to be done with the parameters

name = "ensw"


# Create the config
config = """
name: "ensw_transformer"
data:
    src: "en"
    trg: "sw"
    dev:   "data/ensw/dev.bpe"
    test:  "data/ensw/test.bpe"
    train:  "data/ensw/train.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/ensw/vocab.txt"
    trg_vocab: "data/ensw/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/ensw_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
"""
#.format(name=name,gdrie_path=os.environ["gdrive_path"],source_language=source_language, target_language=target_language)
#.format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_ensw2.yaml".format(name=name),'w') as f:
    f.write(config)

Train the Model
This single line of joeynmt runs the training using the config we made above

In [4]:
!pwd

/home/freshiasackey_gmail_com


In [9]:
%cd ./..

/home/freshiasackey_gmail_com


In [4]:
!cd joeynmt
!pwd

/home/freshiasackey_gmail_com


In [11]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt;pwd; ls; python3 -m joeynmt train ./configs/transformer_ensw2.yaml;ls

/home/freshiasackey_gmail_com/joeynmt
benchmarks.md	    data     joey-small.png  README.md	       setup.py
CODE_OF_CONDUCT.md  docs     LICENSE	     requirements.txt  test
configs		    joeynmt  models	     scripts
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Pas

2020-04-23 22:05:05,960 cfg.name                           : ensw_transformer
2020-04-23 22:05:05,960 cfg.data.src                       : en
2020-04-23 22:05:05,960 cfg.data.trg                       : sw
2020-04-23 22:05:05,961 cfg.data.dev                       : data/ensw/dev.bpe
2020-04-23 22:05:05,961 cfg.data.test                      : data/ensw/test.bpe
2020-04-23 22:05:05,961 cfg.data.train                     : data/ensw/train.bpe
2020-04-23 22:05:05,961 cfg.data.level                     : bpe
2020-04-23 22:05:05,961 cfg.data.lowercase                 : False
2020-04-23 22:05:05,961 cfg.data.max_sent_length           : 100
2020-04-23 22:05:05,961 cfg.data.src_vocab                 : data/ensw/vocab.txt
2020-04-23 22:05:05,961 cfg.data.trg_vocab                 : data/ensw/vocab.txt
2020-04-23 22:05:05,961 cfg.testing.beam_size              : 5
2020-04-23 22:05:05,961 cfg.testing.alpha                  : 1.0
2020-04-23 22:05:05,961 cfg.training.random_seed           : 42
202

2020-04-23 22:08:28,228 Epoch   2 Step:     1100 Batch Loss:     4.948530 Tokens per Sec:    16534, Lr: 0.000300
2020-04-23 22:08:42,851 Epoch   2 Step:     1200 Batch Loss:     4.629645 Tokens per Sec:    16489, Lr: 0.000300
2020-04-23 22:08:57,463 Epoch   2 Step:     1300 Batch Loss:     4.664552 Tokens per Sec:    16742, Lr: 0.000300
2020-04-23 22:09:11,831 Epoch   2 Step:     1400 Batch Loss:     4.592779 Tokens per Sec:    16645, Lr: 0.000300
2020-04-23 22:09:26,380 Epoch   2 Step:     1500 Batch Loss:     4.226472 Tokens per Sec:    16678, Lr: 0.000300
2020-04-23 22:09:36,080 Epoch   2: total training loss 3720.06
2020-04-23 22:09:36,080 EPOCH 3
2020-04-23 22:09:40,988 Epoch   3 Step:     1600 Batch Loss:     4.440054 Tokens per Sec:    16163, Lr: 0.000300
2020-04-23 22:09:55,522 Epoch   3 Step:     1700 Batch Loss:     4.414383 Tokens per Sec:    16729, Lr: 0.000300
2020-04-23 22:10:10,107 Epoch   3 Step:     1800 Batch Loss:     4.320911 Tokens per Sec:    16680, Lr: 0.000300
2

2020-04-23 22:19:42,332 Epoch   7 Step:     4900 Batch Loss:     3.272926 Tokens per Sec:    16822, Lr: 0.000300
2020-04-23 22:19:56,834 Epoch   7 Step:     5000 Batch Loss:     3.183771 Tokens per Sec:    16734, Lr: 0.000300
2020-04-23 22:20:34,446 Hooray! New best validation result [ppl]!
2020-04-23 22:20:34,446 Saving new checkpoint.
2020-04-23 22:20:34,636 Example #0
2020-04-23 22:20:34,636 	Source:     span
2020-04-23 22:20:34,636 	Reference:  shibiri
2020-04-23 22:20:34,636 	Hypothesis: kikoa
2020-04-23 22:20:34,636 Example #1
2020-04-23 22:20:34,636 	Source:     administer
2020-04-23 22:20:34,636 	Reference:  amuru
2020-04-23 22:20:34,636 	Hypothesis: mkosefu
2020-04-23 22:20:34,637 Example #2
2020-04-23 22:20:34,637 	Source:     Then as to Samood, they were destroyed by an excessively severe punishment.
2020-04-23 22:20:34,637 	Reference:  Basi Thamudi waliangamizwa kwa balaa kubwa mno.
2020-04-23 22:20:34,637 	Hypothesis: Basi kama vile vile walivyo wafikia wakhabari kwa adhab

2020-04-23 22:29:39,144 Epoch  11 Step:     8100 Batch Loss:     3.043009 Tokens per Sec:    16602, Lr: 0.000300
2020-04-23 22:29:53,574 Epoch  11 Step:     8200 Batch Loss:     2.711674 Tokens per Sec:    16527, Lr: 0.000300
2020-04-23 22:30:08,143 Epoch  11 Step:     8300 Batch Loss:     2.709927 Tokens per Sec:    16569, Lr: 0.000300
2020-04-23 22:30:22,525 Epoch  11 Step:     8400 Batch Loss:     2.767094 Tokens per Sec:    16389, Lr: 0.000300
2020-04-23 22:30:37,069 Epoch  11 Step:     8500 Batch Loss:     2.919308 Tokens per Sec:    16432, Lr: 0.000300
2020-04-23 22:30:51,530 Epoch  11 Step:     8600 Batch Loss:     2.700101 Tokens per Sec:    16490, Lr: 0.000300
2020-04-23 22:30:52,433 Epoch  11: total training loss 2227.07
2020-04-23 22:30:52,433 EPOCH 12
2020-04-23 22:31:06,273 Epoch  12 Step:     8700 Batch Loss:     2.647313 Tokens per Sec:    16555, Lr: 0.000300
2020-04-23 22:31:20,838 Epoch  12 Step:     8800 Batch Loss:     2.659315 Tokens per Sec:    16648, Lr: 0.000300


2020-04-23 22:40:20,301 Epoch  16 Step:    11900 Batch Loss:     2.571353 Tokens per Sec:    16574, Lr: 0.000300
2020-04-23 22:40:34,880 Epoch  16 Step:    12000 Batch Loss:     2.386926 Tokens per Sec:    16710, Lr: 0.000300
2020-04-23 22:41:00,056 Hooray! New best validation result [ppl]!
2020-04-23 22:41:00,056 Saving new checkpoint.
2020-04-23 22:41:00,239 Example #0
2020-04-23 22:41:00,240 	Source:     span
2020-04-23 22:41:00,240 	Reference:  shibiri
2020-04-23 22:41:00,240 	Hypothesis: kikombe
2020-04-23 22:41:00,240 Example #1
2020-04-23 22:41:00,240 	Source:     administer
2020-04-23 22:41:00,240 	Reference:  amuru
2020-04-23 22:41:00,240 	Hypothesis: ustadi
2020-04-23 22:41:00,240 Example #2
2020-04-23 22:41:00,240 	Source:     Then as to Samood, they were destroyed by an excessively severe punishment.
2020-04-23 22:41:00,240 	Reference:  Basi Thamudi waliangamizwa kwa balaa kubwa mno.
2020-04-23 22:41:00,241 	Hypothesis: Basi walipo angamizwa kwa Haki waliangamizwa kwa adhab

2020-04-23 22:50:00,320 Epoch  20 Step:    15100 Batch Loss:     2.390890 Tokens per Sec:    16529, Lr: 0.000300
2020-04-23 22:50:14,904 Epoch  20 Step:    15200 Batch Loss:     2.334396 Tokens per Sec:    16701, Lr: 0.000300
2020-04-23 22:50:29,267 Epoch  20 Step:    15300 Batch Loss:     2.410015 Tokens per Sec:    16415, Lr: 0.000300
2020-04-23 22:50:43,495 Epoch  20 Step:    15400 Batch Loss:     2.147029 Tokens per Sec:    16267, Lr: 0.000300
2020-04-23 22:50:57,921 Epoch  20 Step:    15500 Batch Loss:     2.312304 Tokens per Sec:    16435, Lr: 0.000300
2020-04-23 22:51:12,357 Epoch  20 Step:    15600 Batch Loss:     2.273867 Tokens per Sec:    16825, Lr: 0.000300
2020-04-23 22:51:19,013 Epoch  20: total training loss 1886.47
2020-04-23 22:51:19,013 EPOCH 21
2020-04-23 22:51:26,991 Epoch  21 Step:    15700 Batch Loss:     2.162075 Tokens per Sec:    16469, Lr: 0.000300
2020-04-23 22:51:41,412 Epoch  21 Step:    15800 Batch Loss:     2.388552 Tokens per Sec:    16468, Lr: 0.000300


2020-04-23 23:00:33,295 Epoch  25 Step:    18900 Batch Loss:     2.188258 Tokens per Sec:    16615, Lr: 0.000300
2020-04-23 23:00:47,712 Epoch  25 Step:    19000 Batch Loss:     2.804485 Tokens per Sec:    16653, Lr: 0.000300
2020-04-23 23:01:14,315 Hooray! New best validation result [ppl]!
2020-04-23 23:01:14,315 Saving new checkpoint.
2020-04-23 23:01:14,495 Example #0
2020-04-23 23:01:14,495 	Source:     span
2020-04-23 23:01:14,495 	Reference:  shibiri
2020-04-23 23:01:14,495 	Hypothesis: kikombe
2020-04-23 23:01:14,495 Example #1
2020-04-23 23:01:14,495 	Source:     administer
2020-04-23 23:01:14,495 	Reference:  amuru
2020-04-23 23:01:14,495 	Hypothesis: msimamizi
2020-04-23 23:01:14,495 Example #2
2020-04-23 23:01:14,496 	Source:     Then as to Samood, they were destroyed by an excessively severe punishment.
2020-04-23 23:01:14,496 	Reference:  Basi Thamudi waliangamizwa kwa balaa kubwa mno.
2020-04-23 23:01:14,496 	Hypothesis: Basi kwa Shamudi waliangamizwa kwa adhabu iliyo chu

2020-04-23 23:10:07,971 Epoch  29 Step:    22100 Batch Loss:     2.003094 Tokens per Sec:    16558, Lr: 0.000300
2020-04-23 23:10:22,307 Epoch  29 Step:    22200 Batch Loss:     2.184560 Tokens per Sec:    16531, Lr: 0.000300
2020-04-23 23:10:36,912 Epoch  29 Step:    22300 Batch Loss:     2.463721 Tokens per Sec:    16552, Lr: 0.000300
2020-04-23 23:10:51,446 Epoch  29 Step:    22400 Batch Loss:     2.145364 Tokens per Sec:    16865, Lr: 0.000300
2020-04-23 23:11:05,798 Epoch  29 Step:    22500 Batch Loss:     2.287599 Tokens per Sec:    16605, Lr: 0.000300
2020-04-23 23:11:20,231 Epoch  29 Step:    22600 Batch Loss:     2.072596 Tokens per Sec:    16695, Lr: 0.000300
2020-04-23 23:11:30,196 Epoch  29: total training loss 1699.23
2020-04-23 23:11:30,197 EPOCH 30
2020-04-23 23:11:34,956 Epoch  30 Step:    22700 Batch Loss:     1.953430 Tokens per Sec:    16315, Lr: 0.000300
2020-04-23 23:11:49,298 Epoch  30 Step:    22800 Batch Loss:     1.949152 Tokens per Sec:    16591, Lr: 0.000300


In [3]:
#copy saved models
!cp -r joeynmt/models/{name}_transformer/* my_sawa_models

In [12]:
#copy saved models2
!mkdir sawa_models
!cp -r joeynmt/models/ensw_transformer/* sawa_models